# System

In [ ]:
if is_debug: print("\n---------- This notenook name: nb_bronze_function_extract_json - Start ----------\n")

In [ ]:
medallion_name = "Bronze"
if is_debug: print(f"medallion_name: {medallion_name}")

In [ ]:
# Current session info
if is_debug: display(ss.getActiveSession())

# Functions

## Create temp view cfg

In [ ]:
if "JSON" in list_technology:
    def fn_create_tvw_cfg(dict_parameter):
        fn_name        = stk()[0][3]
        caller_fn_name = stk()[1].function.replace("<module>", "")
        if is_debug: par = {}

        try:
            sql_code = f"""SELECT
    `sequence`
    , `column_name`
    , `parent_column_name`
    , `data_type`
FROM delta.`{global_parameter.abfs_path_lh_cfg}/Tables/extract_object_json`
WHERE
    `technology`                  = \"{dict_parameter["technology"]}\"
    AND `frequency`               = \"{dict_parameter["frequency"]}\"
    AND IFNULL(`folder_name`, "") = \"{dict_parameter["folder_name"]}\"
    AND IFNULL(`file_name`, "")   = \"{dict_parameter["file_name"]}\"
    AND `is_extracted`            = 1
ORDER BY `sequence`;"""

            rv        = fn_execute_spark_sql(sql_code)
            sdf       = rv[2]
            sdf_count = rv[3]

            sdf.createOrReplaceTempView('tvw_cfg')

            alert             = "Success"
            alert_description = f"tvw_cfg (count): {sdf_count}"

            # fn_print_debug_info_123("Success", fn_name, par) # test "except"

            return (alert, alert_description, sdf, sdf_count)
        except Exception as ex:
            alert             = "Danger"
            alert_description = str(ex)

            return (alert, alert_description, None, None)
        finally:
            if is_debug:
                par["locals"] = locals()
                if "sdf" in locals(): par["sdf"] = sdf.show(n = 5)
                fn_print_debug_info(alert, fn_name, par)
                del par

            fn_local_log_insert(global_parameter.process_timestamp, medallion_name, fn_name, fn_locals_to_json(locals())[2], alert, alert_description)

## Get root array info

In [ ]:
if "JSON" in list_technology:
    def fn_get_root_array_info():
        fn_name        = stk()[0][3]
        caller_fn_name = stk()[1].function.replace("<module>", "")
        if is_debug: par = {}

        try:
            sql_code = f"""SELECT column_name
    FROM tvw_cfg
    WHERE
        parent_column_name = \"root\"
        AND data_type = \"Array\"
            """
            rv               = fn_execute_spark_sql(sql_code)
            sdf_root_array   = rv[2]
            root_array_count = rv[3]

            if root_array_count > 0:
                does_root_array_exist  = True
                root_array_column_name = sdf_root_array.collect()[0][0]
            else:
                does_root_array_exist  = False
                root_array_column_name = ""
                
            alert             = "Success"
            alert_description = f"Root array (exists?, name): {does_root_array_exist}, \"{root_array_column_name}\""

            # fn_print_debug_info_123("Success", fn_name, par) # test "except"

            return (alert, alert_description, does_root_array_exist, root_array_column_name)
        except Exception as ex:
            alert             = "Danger"
            alert_description = str(ex)

            return (alert, alert_description, None, None)
        finally:
            if is_debug:
                par["locals"] = locals()
                if "sdf" in locals(): par["sdf"] = sdf.show(n = 5)
                fn_print_debug_info(alert, fn_name, par)
                del par

            fn_local_log_insert(global_parameter.process_timestamp, medallion_name, fn_name, fn_locals_to_json(locals())[2], alert, alert_description)

## Create temp view parent_child

In [ ]:
if "JSON" in list_technology:
    def fn_create_cfg_parent_child(does_root_array_exist, root_array_column_name):
        fn_name        = stk()[0][3]
        caller_fn_name = stk()[1].function.replace("<module>", "")
        if is_debug: par = {}

        try:
            level = 1

            # Create root array row
            if does_root_array_exist == True:
                sql_code = f"""SELECT
    `sequence`
    , 1                                  AS `level`
    , `column_name`
    , IFNULL(`parent_column_name`, \"\") AS `parent_column_name`
    , ''                                 AS `breadcrumb_column_name`
    , ''                                 AS `breadcrumb_parent_column_name`
    , `data_type`
FROM `tvw_cfg`
WHERE
    `data_type`              = \"Array\"
    AND `parent_column_name` = \"root\";"""
            else:
                sql_code = f"""    SELECT
    0         AS `sequence`
    , 0       AS `level`
    , ''      AS `column_name`
    , 'root'  AS `parent_column_name`
    , ''      AS `breadcrumb_column_name`
    , ''      AS `breadcrumb_parent_column_name`
    , 'Array' AS `data_type`;"""
            
            sdf = fn_execute_spark_sql(sql_code)[2]
            sdf.createOrReplaceTempView('tvw_recursive')

            # Loop recursively to create levels
            while True:
                sql_code = f"""SELECT
    P.`sequence`
    , {level} + 1 AS `level`
    , IFNULL(P.`column_name`       , \"\") AS`column_name`
    , IFNULL(P.`parent_column_name`, \"\") AS `parent_column_name`        
    , CONCAT(
        C.`breadcrumb_column_name`
        , IF(C.`breadcrumb_column_name` = \"\", \"\", \"_\")
        , P.`column_name`
    ) AS `breadcrumb_column_name`
    , IF(
        IFNULL(P.`parent_column_name`, \"\") = \"{root_array_column_name}\"
        , \"\"
        , CONCAT(
            C.`breadcrumb_parent_column_name`
            , IF(C.`breadcrumb_parent_column_name` = \"\", \"\", \"_\")
            , P.`parent_column_name`
        )
    ) AS `breadcrumb_parent_column_name`
    , IFNULL(P.`data_type`         , \"\") AS `data_type`
FROM
`tvw_cfg`            AS P
JOIN `tvw_recursive` AS C ON IFNULL(P.`parent_column_name`, \"\") = C.`column_name`;"""

                sdf_recursive = fn_execute_spark_sql(sql_code)[2]
                sdf_recursive.createOrReplaceTempView('tvw_recursive')
                sdf = sdf.union(sdf_recursive)

                if sdf_recursive.count() == 0:
                    sdf.createOrReplaceTempView("tvw_final")
                    break
                else:
                    level += 1

            # Add column "child_list" and fake root array row
            sql_code = f"""CREATE OR REPLACE TEMP VIEW tvw_cfg_parent_child AS
"""

            if does_root_array_exist == False:
                sql_code += f"""SELECT
    CAST(0 AS Long) AS `sequence`
    , 1             AS `level`
    , 'results'     AS `column_name`
    , 'root'        AS `parent_column_name`
    , ''            AS `breadcrumb_column_name`
    , ''            AS `breadcrumb_parent_column_name`
    , 'Array'       AS `data_type`
    , CAST(collect_list(`column_name`) AS String) AS `child_list`
FROM `tvw_final` AS H
WHERE
    H.`data_type` IN ('Array', 'Struct')
    AND H.`parent_column_name` = ''
UNION ALL """

            sql_code += f"""SELECT
    F.`sequence`
    , F.`level`
    , F.`column_name`
    , IF(F.`parent_column_name` = '', 'results', F.`parent_column_name`) AS `parent_column_name`
    , F.`breadcrumb_column_name`
    , F.`breadcrumb_parent_column_name`
    , F.`data_type`
    , IFNULL(G.`child_list`, '') AS `child_list`
FROM
    `tvw_final` AS F
    LEFT JOIN 
    (
        SELECT
            `parent_column_name`
            , CAST(collect_list(`column_name`) AS String) as `child_list`
        FROM `tvw_cfg`
        WHERE
            `parent_column_name` != 'root'
            AND `data_type` in ('Array', 'Struct')
        GROUP BY `parent_column_name`
    ) AS G
    ON F.`column_name` = G.`parent_column_name`
WHERE F.`sequence` > 0
ORDER BY
    `level`
    , `sequence`;"""

            fn_execute_spark_sql(sql_code)[2]
            sdf_cfg = fn_execute_spark_sql("SELECT * FROM `tvw_cfg_parent_child`;")[2]
            sdf_cfg.createOrReplaceTempView("tvw_cfg")
            sdf_cfg_count = sdf_cfg.count()

            alert             = "Success"
            alert_description = f"Rows (count): {sdf_cfg_count}"

            # fn_print_debug_info_123("Success", fn_name, par) # test "except"

            return (alert, alert_description, sdf_cfg, sdf_cfg_count)
        except Exception as ex:
            alert             = "Danger"
            alert_description = str(ex)

            return (alert, alert_description, None, None)
        finally:
            if is_debug:
                par["locals"] = locals()
                if "sdf" in locals(): par["sdf"] = sdf.show(n = 5)
                fn_print_debug_info(alert, fn_name, par)
                del par

            fn_local_log_insert(global_parameter.process_timestamp, medallion_name, fn_name, fn_locals_to_json(locals())[2], alert, alert_description)

## Get columns

In [ ]:
if "JSON" in list_technology:
    def fn_get_json_column(parent_column_name):
        fn_name        = stk()[0][3]
        caller_fn_name = stk()[1].function.replace("<module>", "")
        if is_debug: par = {}

        try:
            sql_code = f"""SELECT *
FROM tvw_cfg
WHERE `parent_column_name` = \"{parent_column_name}\"
ORDER BY `sequence`;"""

            rv               = fn_execute_spark_sql(sql_code)
            sdf_column       = rv[2]
            sdf_column_count = rv[3]

            alert             = "Success"
            alert_description = f"Rows (count): {sdf_column_count}"

            # fn_print_debug_info_123("Success", fn_name, par) # test "except"

            return (alert, alert_description, sdf_column, sdf_column_count)
        except Exception as ex:
            alert             = "Danger"
            alert_description = str(ex)

            return (alert, alert_description, None, None)
        finally:
            if is_debug:
                par["locals"] = locals()
                if "sdf" in locals(): par["sdf"] = sdf.show(n = 5)
                fn_print_debug_info(alert, fn_name, par)
                del par

            fn_local_log_insert(global_parameter.process_timestamp, medallion_name, fn_name, fn_locals_to_json(locals())[2], alert, alert_description)

## Check if JSON file exists

In [ ]:
if "JSON" in list_technology:
    def fn_does_json_file_exist(dict_parameter):
        fn_name        = stk()[0][3]
        caller_fn_name = stk()[1].function.replace("<module>", "")
        if is_debug: par = {}

        try:
            folder_name = dict_parameter["folder_name_clean"]
            if folder_name != "": folder_name = f"{folder_name}/" # if folder_name is not an empty folder, add slash to the end
            file_name = dict_parameter["file_name_clean"]
            file_name = f"{global_parameter.abfs_path_lh_bronze}/Files/incoming/JSON/{folder_name}{file_name}.json"

            if notebookutils.fs.exists(file_name) == True:
                does_file_exist   = True
                alert             = "Success"
                alert_description = f""
            else:
                does_file_exist = False
                alert             = "Warning"
                alert_description = f"File \"{file_name}\" does not exist."

            # fn_print_debug_info_123("Success", fn_name, par) # test "except"

            return (alert, alert_description, does_file_exist)
        except Exception as ex:
            alert             = "Danger"
            alert_description = str(ex)

            return (alert, alert_description, None)
        finally:
            if is_debug:
                par["locals"] = locals()
                if "sdf" in locals(): par["sdf"] = sdf.show(n = 5)
                fn_print_debug_info(alert, fn_name, par)
                del par

            fn_local_log_insert(global_parameter.process_timestamp, medallion_name, fn_name, fn_locals_to_json(locals())[2], alert, alert_description)

## Read JSON file into temp view json_data

In [ ]:
if "JSON" in list_technology:
    def fn_read_json_file(dict_parameter):
        fn_name        = stk()[0][3]
        caller_fn_name = stk()[1].function.replace("<module>", "")
        if is_debug: par = {}

        try:
            folder_name = dict_parameter["folder_name_clean"]
            if folder_name != "": folder_name = f"{folder_name}/" # if "folder_name" is not an empty folder, add slash to the end

            # Delete tvw_json_data if exists
            sql_code = f"DROP VIEW IF EXISTS `tvw_json_data`;"
            fn_execute_spark_sql(sql_code)[2]

            sql_code = f"""CREATE OR REPLACE TEMP VIEW `tvw_json_data` AS 
SELECT *
FROM json.`{global_parameter.abfs_path_lh_bronze}/Files/incoming/JSON/{folder_name}{dict_parameter["file_name_clean"]}.json`;"""
            
            fn_execute_spark_sql(sql_code)[2]

            alert             = "Success"
            alert_description = f""

            # fn_print_debug_info_123("Success", fn_name, par) # test "except"
            
            return (alert, alert_description)
        except Exception as ex:
            alert             = "Danger"
            alert_description = str(ex)

            return (alert, alert_description)
        finally:
            if is_debug:
                par["locals"] = locals()
                if "sdf" in locals(): par["sdf"] = sdf.show(n = 5)
                fn_print_debug_info(alert, fn_name, par)
                del par

            fn_local_log_insert(global_parameter.process_timestamp, medallion_name, fn_name, fn_locals_to_json(locals())[2], alert, alert_description)

## Create temp view for root array

In [ ]:
if "JSON" in list_technology:
    def fn_create_tvw_root_array(column_name, does_root_array_exist):
        fn_name        = stk()[0][3]
        caller_fn_name = stk()[1].function.replace("<module>", "")
        if is_debug: par = {}

        try:
            # Get child columns for this parent
            sdf_root_array_column = fn_get_json_column(column_name)[2]    

            if does_root_array_exist == True:
                sql_code = f"""/* root array exists */ 
"""
                sql_code += f"""CREATE OR REPLACE TEMP VIEW `tvw_{column_name}` AS 
SELECT 
    CAST(ROW_NUMBER() OVER(ORDER BY (SELECT NULL)) AS Long) AS `fa_{column_name}_id` /* Own ID */ """

                sql_code += f"""
    /* Columns, belonging to root array */"""
                for row in sdf_root_array_column\
                    .sort(sdf_root_array_column.sequence)\
                    .filter((sdf_root_array_column.data_type != "Struct") & (sdf_root_array_column.data_type != "Array"))\
                    .collect():
                    sql_code += f"""
    , CAST(`_{column_name}`.`{row.column_name}` AS {row.data_type}) AS `{row.column_name}` """

                sql_code += f"""
    /* Array and Struct, belonging to root array */ """
                for row in sdf_root_array_column\
                    .sort(sdf_root_array_column.sequence)\
                    .filter((sdf_root_array_column.data_type == "Struct") | (sdf_root_array_column.data_type == "Array"))\
                    .collect():
                    sql_code += f"""
    , `_{column_name}`.`{row.column_name}` AS `{row.column_name}` /* {row.data_type} */ """

                sql_code += f"""
FROM `tvw_json_data` 
LATERAL VIEW EXPLODE_OUTER(`{column_name}`) AS `_{column_name}`;
                """
            else:
                sql_code = f"""/* root array does not exist */ 
"""
                sql_code += f"""CREATE OR REPLACE TEMP VIEW `tvw_{column_name}` AS 
SELECT 
    * 
    , CAST(ROW_NUMBER() OVER(ORDER BY (SELECT NULL)) AS Long) AS `fa_{column_name}_id` /* Own ID */ 
FROM tvw_json_data;"""

            fn_execute_spark_sql(sql_code)

            alert             = "Success"
            alert_description = f""

            # fn_print_debug_info_123("Success", fn_name, par) # test "except"

            return (alert, alert_description, sql_code)
        except Exception as ex:
            alert             = "Danger"
            alert_description = str(ex)

            return (alert, alert_description, None)
        finally:
            if is_debug:
                par["locals"] = locals()
                if "sdf_root_array_column" in locals(): par["sdf_root_array_column"] = sdf_root_array_column.show(n = 5)
                fn_print_debug_info(alert, fn_name, par)
                del par

            fn_local_log_insert(global_parameter.process_timestamp, medallion_name, fn_name, fn_locals_to_json(locals())[2], alert, alert_description)

## Create temp view for not root array

In [ ]:
if "JSON" in list_technology:
    def fn_create_tvw(
        column_name
        , parent_column_name
        , breadcrumb_column_name
        , breadcrumb_parent_column_name
        , data_type
        , child_list
    ):
        fn_name        = stk()[0][3]
        caller_fn_name = stk()[1].function.replace("<module>", "")
        if is_debug: par = {}

        try:
            # Get child columns for this parent
            sdf_column = fn_get_json_column(column_name)[2]
            sdf_column_count = sdf_column.count()

            if breadcrumb_parent_column_name == "": breadcrumb_parent_column_name = parent_column_name

            sql_code = f"""CREATE OR REPLACE TEMP VIEW `tvw_{breadcrumb_column_name}` AS 
SELECT"""
            
            if child_list != "":
                sql_code += f"""
    CAST(ROW_NUMBER() OVER(ORDER BY (SELECT NULL)) AS Long) AS `fa_{breadcrumb_column_name}_id` /* Own ID */ 
    , `fa_{breadcrumb_parent_column_name}_id` /* Parent ID */ """
            else:
                sql_code += f"""
    `fa_{breadcrumb_parent_column_name}_id` /* Parent ID */ """

            if sdf_column_count > 0:
                sql_code += f"""
    /* Columns, belonging to \"{column_name}\" */"""
                for row in sdf_column\
                    .sort(sdf_column.sequence)\
                    .filter((sdf_column.data_type != "Struct") & (sdf_column.data_type != "Array"))\
                    .collect():
                    
                    if data_type == "Array":
                        sql_code += f"""
    , CAST(`_{row.breadcrumb_parent_column_name}`.`{row.column_name}` AS {row.data_type}) AS `{row.breadcrumb_column_name}` """
                    else:
                        sql_code += f"""
    , CAST(`{row.breadcrumb_parent_column_name}`.`{row.column_name}` AS {row.data_type}) AS `{row.breadcrumb_column_name}` """
                
                sql_code += f"""
    /* Array and Struct, belonging to \"{column_name}\" */ """
                for row in sdf_column\
                    .sort(sdf_column.sequence)\
                    .filter((sdf_column.data_type == "Struct") | (sdf_column.data_type == "Array"))\
                    .collect():
                    if data_type == "Array":
                        sql_code += f"""
    , `_{row.breadcrumb_parent_column_name}`.`{row.column_name}` AS `{row.breadcrumb_column_name}` /* {row.data_type} */ """
                    else:
                        sql_code += f"""
    , `{row.breadcrumb_parent_column_name}`.`{row.column_name}` AS `{row.breadcrumb_column_name}` /* {row.data_type} */ """
            else:
                sql_code += f"""
    , `_{breadcrumb_column_name}` """

            sql_code += f"""
FROM `tvw_{breadcrumb_parent_column_name}` """

            if data_type == "Array":
                sql_code += f"""
LATERAL VIEW EXPLODE_OUTER(`{breadcrumb_column_name}`) as `_{breadcrumb_column_name}` /* {data_type} */ """

            sql_code += ";"

            fn_execute_spark_sql(sql_code)[2]

            alert             = "Success"
            alert_description = f""

            # fn_print_debug_info_123("Success", fn_name, par) # test "except"

            return (alert, alert_description, sql_code)
        except Exception as ex:
            alert             = "Danger"
            alert_description = str(ex)

            return (alert, alert_description, None)
        finally:
            if is_debug:
                par["locals"] = locals()
                if "sdf_column" in locals(): par["sdf_column"] = sdf_column.show(n = 5)
                fn_print_debug_info(alert, fn_name, par)
                del par

            fn_local_log_insert(global_parameter.process_timestamp, medallion_name, fn_name, fn_locals_to_json(locals())[2], alert, alert_description)

## Get a list of the JSON files to extract

In [ ]:
if "JSON" in list_technology:
    def fn_get_extract_list(
        technology
        , frequency
    ):
        fn_name        = stk()[0][3]
        caller_fn_name = stk()[1].function.replace("<module>", "")
        if is_debug: par = {}

        try:
            sql_code = f"""SELECT DISTINCT
    `technology`
    , `frequency`
    , IFNULL(`folder_name`, \"\") AS `folder_name`
    , `file_name`
FROM delta.`{global_parameter.abfs_path_lh_cfg}/Tables/extract_object_json`
WHERE
    `technology`       = \"{technology}\"
    AND `frequency`    = \"{frequency}\"
    AND `is_extracted` = 1;"""
            sdf_extract = fn_execute_spark_sql(sql_code)[2]
            sdf_extract_count = sdf_extract.count()

            alert             = "Success"
            alert_description = f"Rows (count): {sdf_extract_count}"

            # fn_print_debug_info_123("Success", fn_name, par) # test "except"

            return (alert, alert_description, sdf_extract, sdf_extract_count)
        except Exception as ex:
            alert             = "Danger"
            alert_description = str(ex)

            return (alert, alert_description, None, None)
        finally:
            if is_debug:
                par["locals"] = locals()
                if "sdf_extract" in locals(): par["sdf_extract"] = sdf_extract.show(n = 5)
                fn_print_debug_info(alert, fn_name, par)
                del par

            fn_local_log_insert(global_parameter.process_timestamp, medallion_name, fn_name, fn_locals_to_json(locals())[2], alert, alert_description)

## Create dynamic code to join the temp views

In [ ]:
if "JSON" in list_technology:
    def fn_get_sql_code_join_tvw():
        fn_name        = stk()[0][3]
        caller_fn_name = stk()[1].function.replace("<module>", "")
        if is_debug: par = {}

        try:
            # Generate SELECT statement
            sql_code = f"""SELECT
    `sequence`
    , IF (
        `breadcrumb_parent_column_name` = ''
        , CONCAT(`parent_column_name`, '`.`', `breadcrumb_column_name`)
        , `breadcrumb_column_name`
    ) AS `column_name`
FROM `tvw_cfg`
WHERE
    data_type != 'Array'
    AND `column_name` NOT IN
    (
        SELECT DISTINCT `parent_column_name`
        FROM `tvw_cfg`
    )

UNION ALL SELECT
    `sequence`
    , CONCAT('_', `breadcrumb_column_name`) AS `column_name`
FROM `tvw_cfg`
WHERE
    data_type = 'Array'
    AND column_name NOT IN
    (
        SELECT DISTINCT `parent_column_name`
        FROM `tvw_cfg`
    )
ORDER BY `sequence`;"""
            sdf = fn_execute_spark_sql(sql_code)[2]

            list_select = str([row.column_name for row in sdf.select('column_name').collect()]).replace("[", "").replace("]", "").replace("'", "`")
            sql_code_select = f"""SELECT DISTINCT {list_select}"""

            # Generate FROM statement
            sql_code = f"""SELECT
    `sequence`
    , CONCAT(
        CONCAT('`tvw_', column_name, '`')
        , ' AS '
        , CONCAT('`', column_name, '`')
    ) AS statement_from
FROM
    (
        SELECT
            ROW_NUMBER() OVER (ORDER BY `sequence`) AS R
            , `sequence`
            , column_name
        FROM `tvw_cfg`
        WHERE data_type IN ('Array', 'Struct')
    ) AS A
WHERE A.R = 1

UNION ALL SELECT
    `sequence`
    , CONCAT(
        ' LEFT JOIN '
        , CONCAT('`tvw_', IF(
            breadcrumb_column_name = ''
            , column_name
            , breadcrumb_column_name), '`'
        )
        , ' AS '
        , IF(
            breadcrumb_column_name = ''
            , CONCAT('`', column_name, '`')
            , CONCAT('`', breadcrumb_column_name, '`')
        )    
        , ' ON '    
        , IF(
            breadcrumb_parent_column_name = ''
            , CONCAT('`', parent_column_name, '`')
            , CONCAT('`', breadcrumb_parent_column_name, '`')
        )        
        , '.'
        , IF(
            breadcrumb_parent_column_name = ''
            , CONCAT('`fa_', parent_column_name, '_id`')
            , CONCAT('`fa_', breadcrumb_parent_column_name, '_id`')
        )
        , ' = '    
        , CONCAT('`', breadcrumb_column_name, '`')
        , '.'
        , IF(
            breadcrumb_parent_column_name = ''
            , CONCAT('`fa_', parent_column_name, '_id`')
            , CONCAT('`fa_', breadcrumb_parent_column_name, '_id`')
        )
    ) AS statement_from
FROM
    (
        SELECT
            ROW_NUMBER() OVER (ORDER BY `sequence`) AS R
            , *
        FROM `tvw_cfg`
        WHERE data_type IN ('Array', 'Struct')
    ) AS A
WHERE A.R > 1
ORDER BY `sequence`;"""
            sdf = fn_execute_spark_sql(sql_code)[2]

            list_from = [row.statement_from for row in sdf.select('statement_from').collect()]

            statement_from = ""
            for row in list_from: statement_from += row
            sql_code_from = f"""FROM {statement_from}"""

            # Generate the final statement
            sql_code_join = f"{sql_code_select}\n{sql_code_from};"

            alert             = "Success"
            alert_description = f""

            # fn_print_debug_info_123("Success", fn_name, par) # test "except"

            return (alert, alert_description, sql_code_join)
        except Exception as ex:
            alert             = "Danger"
            alert_description = str(ex)

            return (alert, alert_description, None)
        finally:
            if is_debug:
                par["locals"] = locals()
                if "sdf" in locals(): par["sdf"] = sdf.show(n = 5)
                fn_print_debug_info(alert, fn_name, par)
                del par

            fn_local_log_insert(global_parameter.process_timestamp, medallion_name, fn_name, fn_locals_to_json(locals())[2], alert, alert_description)

## Clean up column names

In [ ]:
if "JSON" in list_technology:
    def fn_clean_column_name_in_sdf_final(sdf_final):
        fn_name        = stk()[0][3]
        caller_fn_name = stk()[1].function.replace("<module>", "")
        if is_debug: par = {}

        try:
            # Get column names
            list_column = sdf_final.columns
            
            # Create dataframe with column names
            sdf_column = sqlContext.createDataFrame([(l,) for l in list_column], ["column_name"])
            
            # Create temp view with column names
            sdf_column.createOrReplaceTempView("tvw_column")
            
            # Replace first underscore from column name
            sql_code = f"""SELECT
    IF(LEFT(`column_name`, 1) = '_', SUBSTRING(`column_name`, 2, LEN(`column_name`)), `column_name`) AS `column_name`
    , IF(LEFT(`column_name`, 1) = '_', SUBSTRING(`column_name`, 2, LEN(`column_name`)), `column_name`) AS `column_name_clean`
FROM `tvw_column`;"""
            
            # Create dataframe with column names
            sdf_column = fn_execute_spark_sql(sql_code)[2]
            
            # Delete the temp view
            fn_execute_spark_sql("DROP VIEW IF EXISTS tvw_column;")[2]

            # Clean non alphanumericals
            sdf_column = fn_replace_all_not_alphanumericals_in_single_column_in_sdf(
                sdf_column,
                "column_name_clean",
                "_"
            )[2]

            # Clean accents
            sdf_column = sdf_column.withColumn("column_name_clean", fn_clean_accent_in_sdf_column("column_name_clean")[2])

            # Convert to pdf
            pdf_column = sdf_column.toPandas()

            column_list = pdf_column["column_name_clean"].to_list()

            # Rename all the columns
            sdf_final = sdf_final.toDF(*column_list)

            # Count the rows in sdf_final
            sdf_final_count = sdf_final.count()

            alert             = "Success"
            alert_description = f"Rows (count): {sdf_final_count}"

            # fn_print_debug_info_123("Success", fn_name, par) # test "except"

            return (alert, alert_description, sdf_final, sdf_final_count)
        except Exception as ex:
            alert             = "Danger"
            alert_description = str(ex)

            return (alert, alert_description, None, None)
        finally:
            if is_debug:
                par["locals"] = locals()
                if "sdf_final" in locals(): par["sdf_final"] = sdf_final.show(n = 5)
                fn_print_debug_info(alert, fn_name, par)
                del par

            fn_local_log_insert(global_parameter.process_timestamp, medallion_name, fn_name, fn_locals_to_json(locals())[2], alert, alert_description)

## Save the result in bronze table

In [ ]:
if "JSON" in list_technology:
    def fn_save_json_as_table(dict_parameter, sdf):
        fn_name        = stk()[0][3]
        caller_fn_name = stk()[1].function.replace("<module>", "")
        if is_debug: par = {}

        try:
            technology  = dict_parameter["technology_clean"]
            folder_name = dict_parameter["folder_name_clean"]
            file_name   = dict_parameter["file_name_clean"]

            # Generate full table path
            file_name = f"{global_parameter.abfs_path_lh_bronze}/Tables/{technology}_{folder_name}_{file_name}"

            # Remove double underscore (folder_name is empty)
            file_name = file_name.replace(f"__", "_")

            # Save the extracted JSON file as delta table in lh_bronze
            fn_save_sdf_as_table(
                sdf
                , "delta"
                , "overwrite"
                , file_name
                , dict_parameter
            )

            alert             = "Success"
            alert_description = f""

            # fn_print_debug_info_123("Success", fn_name, par) # test "except"

            return (alert, alert_description, file_name)
        except Exception as ex:
            alert             = "Danger"
            alert_description = str(ex)

            return (alert, alert_description, None)
        finally:
            if is_debug:
                par["locals"] = locals()
                if "sdf" in locals(): par["sdf"] = sdf.show(n = 5)
                fn_print_debug_info(alert, fn_name, par)
                del par

            fn_local_log_insert(global_parameter.process_timestamp, medallion_name, fn_name, fn_locals_to_json(locals())[2], alert, alert_description)

## Drop all temp views

In [ ]:
if "JSON" in list_technology:
    def fn_drop_all_temp_views():
        fn_name        = stk()[0][3]
        caller_fn_name = stk()[1].function.replace("<module>", "")
        if is_debug: par = {}

        try:
            sdf = fn_execute_spark_sql(f"SHOW VIEWS;")[2]

            for row in sdf.filter(sdf.isTemporary == True).collect():
                sql_code = f"DROP VIEW IF EXISTS `{row.viewName}`;"
                if is_debug: print(f"sql_code: {sql_code}")
                fn_execute_spark_sql(sql_code)[2]

            alert             = "Success"
            alert_description = f""

            # fn_print_debug_info_123("Success", fn_name, par) # test "except"

            return (alert, alert_description)
        except Exception as ex:
            alert             = "Danger"
            alert_description = str(ex)

            return (alert, alert_description)
        finally:
            if is_debug:
                par["locals"] = locals()
                if "sdf" in locals(): par["sdf"] = sdf.show(n = 5)
                fn_print_debug_info(alert, fn_name, par)
                del par

            fn_local_log_insert(global_parameter.process_timestamp, medallion_name, fn_name, fn_locals_to_json(locals())[2], alert, alert_description)

# Operations

## Extract JSON

In [ ]:
if "JSON" in list_technology:
    def fn_extract_json(technology, frequency):
        fn_name        = stk()[0][3]
        caller_fn_name = stk()[1].function.replace("<module>", "")
        if is_debug: par = {}

        try:
            # Rename the existing files in lh_bronze
            sdf_renamed = fn_rename_files_in_lh_bronze(technology, frequency, "Archive")[2]
            if is_debug:
                print(f"sdf_renamed")
                display(sdf_renamed)

            # Get a list of JSON files to extract
            sdf_extract = fn_get_extract_list(technology, frequency)[2]
            sdf_extract_count = sdf_extract.count()

            if is_debug:
                print(f"sdf_extract")
                display(sdf_extract)
                print(f"sdf_extract_count: {sdf_extract_count}")

            # Loop sdf_extract
            for row_e in sdf_extract.collect():
                # Set dict_parameter
                dict_parameter = {"technology": row_e.technology, "frequency": row_e.frequency, "folder_name": row_e.folder_name, "file_name": row_e.file_name}                

                # Clean up "technology", "folder_name", "file_name" - Start
                # Clean accents and not alphanumerics
                technology_clean  = fn_clean_not_alphanumeric_in_string(fn_clean_accent_in_string(technology)[2], "_")[2]
                folder_name_clean = row_e.folder_name.replace("/", "ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789") # keep "/" while cleaning
                folder_name_clean = fn_clean_not_alphanumeric_in_string(fn_clean_accent_in_string(folder_name_clean)[2], "_")[2]
                folder_name_clean = folder_name_clean.replace("ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789", "/")
                file_name_clean   = fn_clean_not_alphanumeric_in_string(fn_clean_accent_in_string(row_e.file_name)[2], "_")[2]

                dict_parameter["technology_clean"]  = technology_clean
                dict_parameter["folder_name_clean"] = folder_name_clean
                dict_parameter["file_name_clean"]   = file_name_clean

                if is_debug:
                    print(f"folder_name_clean: >{folder_name_clean}<")
                    print(f"file_name_clean: >{file_name_clean}<")
                    print(f"dict_parameter: {dict_parameter}")
                # Clean up "technology", "folder_name", "file_name" - End

                # Check if the current file exists
                rv              = fn_does_json_file_exist(dict_parameter)
                does_file_exist = rv[2]
                if is_debug: print(f"does_file_exist: {does_file_exist}")

                if does_file_exist == True:
                    # Create tvw_cfg
                    fn_create_tvw_cfg(dict_parameter)
                    
                    # Check if root array exists and get its column_name
                    rv = fn_get_root_array_info()    
                    does_root_array_exist = rv[2]
                    root_array_column_name = rv[3]

                    if is_debug:
                        print(f"does_root_array_exist: {does_root_array_exist}")
                        print(f"root_array_column_name: >{root_array_column_name}<")

                    # Create tvw_parent_child - Start
                    rv = fn_create_cfg_parent_child(does_root_array_exist, root_array_column_name)
                    sdf_cfg = rv[2]
                    sdf_cfg_count = rv[3]
                    if is_debug:
                        print("sdf_cfg:")
                        display(sdf_cfg)
                        print(f"sdf_cfg_count: {sdf_cfg_count}")

                    # Clean up
                    fn_execute_spark_sql("DROP VIEW IF EXISTS tvw_cfg_parent_child;")
                    fn_execute_spark_sql("DROP VIEW IF EXISTS tvw_recursive;")
                    fn_execute_spark_sql("DROP VIEW IF EXISTS tvw_final;")
                    # Create tvw_parent_child - End

                    # Read the data from the current file (if exists) into tvw_json_data
                    fn_read_json_file(dict_parameter)

                    # Create temp views for root array and all the nested Array or Struct
                    for row_f in sdf_cfg\
                        .filter((sdf_cfg.data_type == "Struct") | (sdf_cfg.data_type == "Array"))\
                        .sort(sdf_cfg.level, sdf_cfg.sequence)\
                        .collect():

                        if row_f.level == 1:
                            if is_debug:
                                print(f"row_f:")
                                print(f"  level: {row_f.level}")
                                print(f"  sequence: {row_f.sequence}")
                                print(f"  column_name: {row_f.column_name}")
                                print(f"  parent_column_name: {row_f.parent_column_name}")
                                print(f"  breadcrumb_column_name: {row_f.breadcrumb_column_name}")
                                print(f"  breadcrumb_parent_column_name: {row_f.breadcrumb_parent_column_name}")
                                print(f"  data_type: {row_f.data_type}")
                                print(f"  child_list: {row_f.child_list}")
                            
                            fn_create_tvw_root_array(
                                row_f.column_name
                                , does_root_array_exist
                            )
                        else:
                            if is_debug:
                                print(f"row_f:")
                                print(f"  level: {row_f.level}")
                                print(f"  sequence: {row_f.sequence}")
                                print(f"  column_name: {row_f.column_name}")
                                print(f"  parent_column_name: {row_f.parent_column_name}")
                                print(f"  breadcrumb_column_name: {row_f.breadcrumb_column_name}")
                                print(f"  breadcrumb_parent_column_name: {row_f.breadcrumb_parent_column_name}")
                                print(f"  data_type: {row_f.data_type}")
                                print(f"  child_list: {row_f.child_list}")
                            
                            fn_create_tvw(
                                row_f.column_name
                                , row_f.parent_column_name
                                , row_f.breadcrumb_column_name
                                , row_f.breadcrumb_parent_column_name
                                , row_f.data_type
                                , row_f.child_list
                            )
                        if is_debug: print("---------- Loop level 1, >1 end ----------")
                        
                    if is_debug:
                        print("SHOW VIEWS:")
                        display(spark.sql("SHOW VIEWS;"))

                    # Generate SQL code to join the root array to all the nested Array or Struct
                    rv            = fn_get_sql_code_join_tvw()
                    sql_code_join = rv[2]
                    if is_debug: print(f"sql_code_join: {sql_code_join}")
                    
                    # Get the final dataframe
                    rv              = fn_execute_spark_sql(sql_code_join)
                    sdf_final       = rv[2]
                    sdf_final_count = rv[3]

                    # Clean up all the columns names in the final dataframe (delta table does not accept accents and non aplhanumericals in the column names)
                    sdf_final = fn_clean_column_name_in_sdf_final(sdf_final)[2]
                    if is_debug:
                        print(f"sdf_final:")
                        display(sdf_final)
                        print(f"sdf_final_count: {sdf_final_count}")

                    # Save JSON file as table in lh_bronze
                    fn_save_json_as_table(dict_parameter, sdf_final)

                    # Copy and delete JSON file to "lh_bronze/Processed/JSON" (archive) - Start
                    # Set "src_file_path" and "dstn_file_path"
                    now_year  = fn_get_now("year")[1]
                    now_month = fn_get_now("month")[1]
                    src_file_path  = f"{global_parameter.abfs_path_lh_bronze}/Files/incoming/{technology_clean}/{folder_name_clean}/{file_name_clean}.json"
                    dstn_file_path = f"{global_parameter.abfs_path_lh_bronze}/Files/processed/{technology_clean}/{folder_name_clean}/{file_name_clean}/\
{now_year}/{now_month}/\
{file_name_clean}_{global_parameter.process_timestamp}.json"

                    # Remove double slash from source and destination file paths ("folder_name" is empty)
                    src_file_path  = src_file_path.replace("abfss://", "ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789")
                    src_file_path  = src_file_path.replace("//", "/")
                    src_file_path  = src_file_path.replace("ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789", "abfss://")
                    dstn_file_path = dstn_file_path.replace("abfss://", "ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789")
                    dstn_file_path = dstn_file_path.replace("//", "/")
                    dstn_file_path = dstn_file_path.replace("ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789", "abfss://")

                    # Copy and delete
                    copy_file_status = fn_copy_file(src_file_path, dstn_file_path, dict_parameter)[0]
                    if copy_file_status == "Success": fn_delete_file(src_file_path, dict_parameter)

                    if is_debug:
                        print(f"src_file_path:  {src_file_path}")
                        print(f"dstn_file_path: {dstn_file_path}")            
                    # Copy and delete JSON file to "lh_bronze/Processed/JSON" (archive) - End

                    # Clean up (drop all temp views)
                    fn_drop_all_temp_views()

                    # Insert "Success" in lh_log (for each file) - Start
                    # Always log "Success" for extracted objects.
                    alert             = "Success"
                    alert_description = f"Row (count): {sdf_final_count}"

                    if is_debug: del par
                    fn_local_log_insert(global_parameter.process_timestamp, medallion_name, fn_name, fn_locals_to_json(locals())[2], alert, alert_description)
                    # Insert "Success" in lh_log (for each file) - End

                    if is_debug: print("------------------------- Loop end -------------------------")

            alert             = "Success"
            alert_description = ""

            return (alert, alert_description)
        except Exception as ex:
            alert             = "Danger"
            alert_description = str(ex)

            return (alert, alert_description)

            if is_debug:
                par["locals"] = locals()
                fn_print_debug_info(alert, fn_name, par)
                del par
            
            fn_local_log_insert(global_parameter.process_timestamp, medallion_name, fn_name, fn_locals_to_json(locals())[2], alert, alert_description)

In [ ]:
if is_debug: print("\n---------- This notenook name: nb_bronze_function_extract_json - End ----------\n")